In [1]:

import argparse
import os
import time

import imageio
import numpy as np
import torch
import torchvision
from einops import rearrange
from rich import print, pretty

pretty.install()

# Distributed env defaults (torchrun will override)
os.environ.setdefault("MASTER_ADDR", "localhost")
os.environ.setdefault("MASTER_PORT", "29506")

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
from fastvideo.configs.pipelines.wan import Wan2_2_I2V_A14B_Config
from fastvideo.distributed import get_local_torch_device
from fastvideo.distributed.parallel_state import (
    maybe_init_distributed_environment_and_model_parallel,
)
from fastvideo.fastvideo_args import ExecutionMode, FastVideoArgs, TrainingArgs, WorkloadType
from fastvideo.logger import init_logger
from fastvideo.models.schedulers.scheduling_flow_unipc_multistep import (
    FlowUniPCMultistepScheduler,
)
from fastvideo.pipelines import build_pipeline
from fastvideo.pipelines.pipeline_batch_info import ForwardBatch, TrainingBatch
from fastvideo.utils import maybe_download_model
from fastvideo.training.wan_i2v_training_pipeline import WanI2VTrainingPipeline
from fastvideo.models.schedulers.scheduling_flow_match_euler_discrete import (
    FlowMatchEulerDiscreteScheduler,
)
from fastvideo.platforms import current_platform
logger = init_logger(__name__)

INFO 02-06 15:26:48.724 [__init__.py:109] ROCm platform is unavailable: No module named 'amdsmi'
WARNING 02-06 15:26:48.726 [logger.py:122]  By default, logger.info(..) will only log from the local main process. Set logger.info(..., is_local_main_process=False) to log from all processes.
INFO 02-06 15:26:48.731 [__init__.py:47] CUDA is available


In [3]:
MODEL_ID = "Wan-AI/Wan2.2-I2V-A14B-Diffusers"
# LOCAL_DIR = os.path.join("dev", MODEL_ID.replace("/", "-"))
LOCAL_DIR = os.path.join(os.path.expanduser("~"), "dev/data/", MODEL_ID.replace("/", "-"))
OUTPUT_DIR = "video_samples_wan2_2_14B_i2v"

EXAMPLE_PROMPT = (
    "Summer beach vacation style, a white cat wearing sunglasses sits on a surfboard. "
    "The fluffy-furred feline gazes directly at the camera with a relaxed expression. "
    "Blurred beach scenery forms the background featuring crystal-clear waters, distant "
    "green hills, and a blue sky dotted with white clouds. The cat assumes a naturally "
    "relaxed posture, as if savoring the sea breeze and warm sunlight. A close-up shot "
    "highlights the feline's intricate details and the refreshing atmosphere of the seaside."
)
EXAMPLE_IMAGE = "https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/wan_i2v_input.JPG"
NEGATIVE_PROMPT = (
    "色调艳丽，过曝，静态，细节模糊不清，字幕，风格，作品，画作，画面，静止，整体发灰，最差质量，"
    "低质量，JPEG压缩残留，丑陋的，残缺的，多余的手指，画得不好的手部，画得不好的脸部，畸形的，毁容的，"
    "形态畸形的肢体，手指融合，静止不动的画面，杂乱的背景，三条腿，背景人很多，倒着走"
)
def _resolve_model_path() -> str:
    logger.info("Downloading/loading model: %s", MODEL_ID)
    model_path = maybe_download_model(MODEL_ID, local_dir=LOCAL_DIR)
    logger.info("Model path: %s", model_path)
    return model_path


print(LOCAL_DIR)

/home/builder/dev/data/Wan-AI-Wan2.2-I2V-A14B-Diffusers

In [10]:
model_path = _resolve_model_path()

pipeline_config = Wan2_2_I2V_A14B_Config()
fastvideo_args = FastVideoArgs(
    model_path=model_path,
    num_gpus=1,
    tp_size=1,
    sp_size=1,
    hsdp_shard_dim=1,
    hsdp_replicate_dim=1,
    dit_cpu_offload=False,
    text_encoder_cpu_offload=True,
    vae_cpu_offload=False,
    pipeline_config=pipeline_config,
)
print(fastvideo_args)

INFO 02-06 15:31:13.022 [1682206808.py:21] Downloading/loading model: Wan-AI/Wan2.2-I2V-A14B-Diffusers
INFO 02-06 15:31:13.024 [utils.py:517] Downloading model snapshot from HF Hub for Wan-AI/Wan2.2-I2V-A14B-Diffusers...


Fetching 50 files:   0%|          | 0/50 [00:00<?, ?it/s]

INFO 02-06 15:31:13.148 [utils.py:524] Downloaded model to /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers
INFO 02-06 15:31:13.148 [1682206808.py:23] Model path: /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers


FastVideoArgs(
    model_path='/home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers',
    mode=<ExecutionMode.INFERENCE: 'inference'>,
    workload_type=<WorkloadType.T2V: 't2v'>,
    cache_strategy='none',
    distributed_executor_backend='mp',
    ray_placement_group=None,
    ray_runtime_env=None,
    inference_mode=True,
    trust_remote_code=False,
    revision=None,
    num_gpus=1,
    tp_size=1,
    sp_size=1,
    hsdp_replicate_dim=1,
    hsdp_shard_dim=1,
    dist_timeout=None,
    pipeline_config=Wan2_2_I2V_A14B_Config(
        model_path='',
        pipeline_config_path=None,
        embedded_cfg_scale=6.0,
        flow_shift=5.0,
        flow_shift_sr=None,
        disable_autocast=False,
        is_causal=False,
        dit_config=WanVideoConfig(
            arch_config=WanVideoArchConfig(
                stacked_params_mapping=[],
                _fsdp_shard_conditions=[<function is_blocks at 0x7fb304c93910>],
                _compile_conditions=[<function is_blocks at 0x7fb304c93910>],
                param_names_mapping={
                    '^patch_embedding\\.(.*)$': 'patch_embedding.proj.\\1',
                    '^condition_embedder\\.text_embedder\\.linear_1\\.(.*)$': 
'condition_embedder.text_embedder.fc_in.\\1',
                    '^condition_embedder\\.text_embedder\\.linear_2\\.(.*)$': 
'condition_embedder.text_embedder.fc_out.\\1',
                    '^condition_embedder\\.time_embedder\\.linear_1\\.(.*)$': 
'condition_embedder.time_embedder.mlp.fc_in.\\1',
                    '^condition_embedder\\.time_embedder\\.linear_2\\.(.*)$': 
'condition_embedder.time_embedder.mlp.fc_out.\\1',
                    '^condition_embedder\\.time_proj\\.(.*)$': 'condition_embedder.time_modulation.linear.\\1',
                    '^condition_embedder\\.image_embedder\\.ff\\.net\\.0\\.proj\\.(.*)$': 
'condition_embedder.image_embedder.ff.fc_in.\\1',
                    '^condition_embedder\\.image_embedder\\.ff\\.net\\.2\\.(.*)$': 
'condition_embedder.image_embedder.ff.fc_out.\\1',
                    '^blocks\\.(\\d+)\\.attn1\\.to_q\\.(.*)$': 'blocks.\\1.to_q.\\2',
                    '^blocks\\.(\\d+)\\.attn1\\.to_k\\.(.*)$': 'blocks.\\1.to_k.\\2',
                    '^blocks\\.(\\d+)\\.attn1\\.to_v\\.(.*)$': 'blocks.\\1.to_v.\\2',
                    '^blocks\\.(\\d+)\\.attn1\\.to_out\\.0\\.(.*)$': 'blocks.\\1.to_out.\\2',
                    '^blocks\\.(\\d+)\\.attn1\\.norm_q\\.(.*)$': 'blocks.\\1.norm_q.\\2',
                    '^blocks\\.(\\d+)\\.attn1\\.norm_k\\.(.*)$': 'blocks.\\1.norm_k.\\2',
                    '^blocks\\.(\\d+)\\.attn2\\.to_out\\.0\\.(.*)$': 'blocks.\\1.attn2.to_out.\\2',
                    '^blocks\\.(\\d+)\\.ffn\\.net\\.0\\.proj\\.(.*)$': 'blocks.\\1.ffn.fc_in.\\2',
                    '^blocks\\.(\\d+)\\.ffn\\.net\\.2\\.(.*)$': 'blocks.\\1.ffn.fc_out.\\2',
                    '^blocks\\.(\\d+)\\.norm2\\.(.*)$': 'blocks.\\1.self_attn_residual_norm.norm.\\2'
                },
                reverse_param_names_mapping={},
                lora_param_names_mapping={
                    '^blocks\\.(\\d+)\\.self_attn\\.q\\.(.*)$': 'blocks.\\1.attn1.to_q.\\2',
                    '^blocks\\.(\\d+)\\.self_attn\\.k\\.(.*)$': 'blocks.\\1.attn1.to_k.\\2',
                    '^blocks\\.(\\d+)\\.self_attn\\.v\\.(.*)$': 'blocks.\\1.attn1.to_v.\\2',
                    '^blocks\\.(\\d+)\\.self_attn\\.o\\.(.*)$': 'blocks.\\1.attn1.to_out.0.\\2',
                    '^blocks\\.(\\d+)\\.cross_attn\\.q\\.(.*)$': 'blocks.\\1.attn2.to_q.\\2',
                    '^blocks\\.(\\d+)\\.cross_attn\\.k\\.(.*)$': 'blocks.\\1.attn2.to_k.\\2',
                    '^blocks\\.(\\d+)\\.cross_attn\\.v\\.(.*)$': 'blocks.\\1.attn2.to_v.\\2',
                    '^blocks\\.(\\d+)\\.cross_attn\\.o\\.(.*)$': 'blocks.\\1.attn2.to_out.0.\\2',
                    '^blocks\\.(\\d+)\\.ffn\\.0\\.(.*)$': 'blocks.\\1.ffn.fc_in.\\2',
                    '^blocks\\.(\\d+)\\.ffn\\.2\\.(.*)$': 'blocks.\\1.ffn.fc_out.\\2'
          

In [12]:
pipeline = build_pipeline(fastvideo_args)

INFO 02-06 15:33:16.902 [utils.py:512] Model already exists locally at /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers
INFO 02-06 15:33:16.903 [__init__.py:42] Model path: /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers
INFO 02-06 15:33:16.903 [__init__.py:44] Building pipeline of type: basic
INFO 02-06 15:33:16.903 [profiler.py:191] Torch profiler disabled; returning no-op controller
INFO 02-06 15:33:16.904 [composed_pipeline_base.py:88] Loading pipeline modules...
INFO 02-06 15:33:16.904 [utils.py:512] Model already exists locally at /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers
INFO 02-06 15:33:16.905 [composed_pipeline_base.py:231] Model path: /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers
INFO 02-06 15:33:16.906 [utils.py:600] Diffusers version: 0.35.0.dev0
INFO 02-06 15:33:16.906 [composed_pipeline_base.py:291] Loading pipeline modules from config: {'_class_name': 'WanImageToVideoPipeline', '_diffusers_version': '0.35.0.dev0', 'boundary_ratio': 0.9, 'image_encoder'

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 02-06 15:33:22.074 [component_loader.py:386] Loading weights took 5.14 seconds
INFO 02-06 15:33:55.251 [composed_pipeline_base.py:370] Loaded module text_encoder from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/text_encoder
INFO 02-06 15:33:55.252 [component_loader.py:995] Loading tokenizer using transformers from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/tokenizer
INFO 02-06 15:33:55.252 [component_loader.py:506] Loading tokenizer from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/tokenizer
INFO 02-06 15:33:55.703 [component_loader.py:554] Loaded tokenizer: T5TokenizerFast
INFO 02-06 15:33:55.704 [composed_pipeline_base.py:370] Loaded module tokenizer from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/tokenizer
INFO 02-06 15:33:55.704 [component_loader.py:995] Loading transformer using diffusers from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer
INFO 02-06 15:33:55.705 [component_loader.py:745] transformer cls_name: WanTransformer3DModel

Loading safetensors checkpoint shards:   0% Completed | 0/12 [00:00<?, ?it/s]


INFO 02-06 15:34:05.581 [component_loader.py:837] Loaded model with 14.29B parameters
INFO 02-06 15:34:50.036 [composed_pipeline_base.py:370] Loaded module transformer from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer
INFO 02-06 15:34:50.037 [component_loader.py:995] Loading transformer_2 using diffusers from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer_2
INFO 02-06 15:34:50.037 [component_loader.py:745] transformer cls_name: WanTransformer3DModel
INFO 02-06 15:34:50.038 [component_loader.py:793] Loading model from 12 safetensors files: ['/home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer_2/diffusion_pytorch_model-00008-of-00012.safetensors', '/home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer_2/diffusion_pytorch_model-00012-of-00012.safetensors', '/home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer_2/diffusion_pytorch_model-00001-of-00012.safetensors', '/home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer

Loading safetensors checkpoint shards:   0% Completed | 0/12 [00:00<?, ?it/s]


INFO 02-06 15:35:20.359 [component_loader.py:837] Loaded model with 14.29B parameters
INFO 02-06 15:36:03.937 [composed_pipeline_base.py:370] Loaded module transformer_2 from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/transformer_2
INFO 02-06 15:36:03.938 [component_loader.py:995] Loading vae using diffusers from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/vae
INFO 02-06 15:36:05.172 [composed_pipeline_base.py:370] Loaded module vae from /home/builder/dev/Wan-AI-Wan2.2-I2V-A14B-Diffusers/vae
INFO 02-06 15:36:05.173 [lora_pipeline.py:147] trainable_transformer_modules: dict_keys(['transformer', 'transformer_2'])
INFO 02-06 15:36:05.173 [__init__.py:59] Pipelines instantiated


In [19]:
pipeline.post_init()

INFO 02-06 15:39:36.388 [composed_pipeline_base.py:159] Creating pipeline stages...


In [20]:
height = 480
width = 832
num_frames = 21
num_inference_steps = 40
guidance_scale = 5.0
seed = 42
output_path = "output-wan14b.mp4"
height_latents = height // 8
width_latents = width // 8
num_latent_frames = (num_frames - 1) // 4 + 1
n_tokens = num_latent_frames * height_latents * width_latents


batch = ForwardBatch(
    data_type="video",
    prompt="a dragon flying in the sky",
    negative_prompt=NEGATIVE_PROMPT,
    image_path="/home/builder/dev/lingbot-world/examples/00/image.jpg",
    height=height,
    width=width,
    num_frames=num_frames,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    seed=seed,
    n_tokens=n_tokens,
    output_path=output_path,
    save_video=True,
    fps=16,
)

In [21]:
# with torch.no_grad():
#     output_batch = pipeline.forward(batch, fastvideo_args)
# torch.cuda.empty_cache()

with torch.no_grad():
    for stage in pipeline.stages:
        print(type(stage))
        batch = stage(batch, fastvideo_args)
output_batch = batch
torch.cuda.empty_cache()

<class 'fastvideo.pipelines.stages.input_validation.InputValidationStage'>

<class 'fastvideo.pipelines.stages.text_encoding.TextEncodingStage'>

<class 'fastvideo.pipelines.stages.conditioning.ConditioningStage'>

<class 'fastvideo.pipelines.stages.timestep_preparation.TimestepPreparationStage'>

<class 'fastvideo.pipelines.stages.latent_preparation.LatentPreparationStage'>

<class 'fastvideo.pipelines.stages.image_encoding.ImageVAEEncodingStage'>

<class 'fastvideo.pipelines.stages.denoising.DenoisingStage'>

  0%|          | 0/40 [00:00<?, ?it/s]

INFO 02-06 15:39:47.133 [wanvideo.py:696] Padding not applied
INFO 02-06 15:40:13.741 [wanvideo.py:696] Padding not applied


<class 'fastvideo.pipelines.stages.decoding.DecodingStage'>

In [16]:
samples = output_batch.output
videos = rearrange(samples, "b c t h w -> t b c h w")
frames = []
for x in videos:
    x = torchvision.utils.make_grid(x, nrow=6)
    x = x.transpose(0, 1).transpose(1, 2).squeeze(-1)
    frames.append((x * 255).numpy().astype(np.uint8))
imageio.mimsave(output_path, frames, fps=batch.fps, format="mp4")
logger.info("Saved video to: %s", output_path)

RuntimeError: Tensor type unknown to einops <class 'NoneType'>

In [8]:
print(type(pipeline))
print(type(pipeline.modules["vae"]))
print(type(pipeline.stages), pipeline.stages)
del fastvideo_args
torch.cuda.empty_cache()

<class 'fastvideo.pipelines.basic.wan.wan_i2v_pipeline.WanImageToVideoPipeline'>

<class 'fastvideo.models.vaes.wanvae.AutoencoderKLWan'>

<class 'list'>
[
    <fastvideo.pipelines.stages.input_validation.InputValidationStage object at 0x7fb2b9e6f9d0>,
    <fastvideo.pipelines.stages.text_encoding.TextEncodingStage object at 0x7fb5ccf1b400>,
    <fastvideo.pipelines.stages.conditioning.ConditioningStage object at 0x7fb290264190>,
    <fastvideo.pipelines.stages.timestep_preparation.TimestepPreparationStage object at 0x7fb7d40616f0>,
    <fastvideo.pipelines.stages.latent_preparation.LatentPreparationStage object at 0x7fb7d40624a0>,
    <fastvideo.pipelines.stages.image_encoding.ImageVAEEncodingStage object at 0x7fb7d4062a40>,
    <fastvideo.pipelines.stages.denoising.DenoisingStage object at 0x7fb7d40602e0>,
    <fastvideo.pipelines.stages.decoding.DecodingStage object at 0x7fb7d40615d0>
]